In [1]:
import os
from os import listdir
import pandas as pd 
import numpy as np 
import matplotlib.pyplot as plt 
import plotly.express as px
import seaborn as sns
from pathlib import Path

import glob
import random
import cv2
import numpy as np
from sklearn.model_selection import train_test_split
from torch.utils.data import DataLoader, Dataset  # Ensure Dataset is imported
import torchvision.transforms as transforms
import torch
import torch.nn as nn
import torch.optim as optim

import cv2
from matplotlib.image import imread

import tensorflow as tf
#from keras.utils.np_utils import to_categorical
from keras.preprocessing import image
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix

print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
import glob
import PIL
import random

random.seed(100)

2025-04-23 06:06:21.330150: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1745388381.532659      31 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1745388381.586524      31 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


Num GPUs Available:  1


In [2]:
breast_img_paths = glob.glob('/kaggle/input/breakhisupd/archive (7)/BreaKHis_v1/BreaKHis_v1/histology_slides/breast/**/*.png', recursive = True)
for img_path in breast_img_paths[:5]:
    print(img_path)

/kaggle/input/breakhisupd/archive (7)/BreaKHis_v1/BreaKHis_v1/histology_slides/breast/benign/SOB/adenosis/SOB_B_A_14-29960CD/200X/SOB_B_A-14-29960CD-200-012.png
/kaggle/input/breakhisupd/archive (7)/BreaKHis_v1/BreaKHis_v1/histology_slides/breast/benign/SOB/adenosis/SOB_B_A_14-29960CD/200X/SOB_B_A-14-29960CD-200-009.png
/kaggle/input/breakhisupd/archive (7)/BreaKHis_v1/BreaKHis_v1/histology_slides/breast/benign/SOB/adenosis/SOB_B_A_14-29960CD/200X/SOB_B_A-14-29960CD-200-006.png
/kaggle/input/breakhisupd/archive (7)/BreaKHis_v1/BreaKHis_v1/histology_slides/breast/benign/SOB/adenosis/SOB_B_A_14-29960CD/200X/SOB_B_A-14-29960CD-200-011.png
/kaggle/input/breakhisupd/archive (7)/BreaKHis_v1/BreaKHis_v1/histology_slides/breast/benign/SOB/adenosis/SOB_B_A_14-29960CD/200X/SOB_B_A-14-29960CD-200-004.png


In [3]:
from pathlib import Path

# Initialize lists
benign = []
malignant = []

# Subtype lists
A, F, PT, TA = [], [], [], []  # benign
DC, LC, MC, PC = [], [], [], []  # malignant

for img in breast_img_paths:
    img_path = Path(img)
    cancer_type = img_path.parts[9].lower()        # 'benign' or 'malignant'
    subtype = img_path.parts[11].lower()            # e.g., 'adenosis', 'fibroadenoma', etc.

    # Classify based on subtype
    if subtype == 'adenosis':
        A.append(img)
    elif subtype == 'fibroadenoma':
        F.append(img)
    elif subtype == 'phyllodes_tumor':
        PT.append(img)
    elif subtype == 'tubular_adenoma':
        TA.append(img)
    elif subtype == 'ductal_carcinoma':
        DC.append(img)
    elif subtype == 'lobular_carcinoma':
        LC.append(img)
    elif subtype == 'mucinous_carcinoma':
        MC.append(img)
    elif subtype == 'papillary_carcinoma':
        PC.append(img)

    # Categorize as benign or malignant
    if cancer_type == 'benign':
        benign.append(img)
    elif cancer_type == 'malignant':
        malignant.append(img)


In [4]:
print("Total examples: ", len(breast_img_paths))
print("Number of benign examples", len(benign))
print("Number of malignant examples", len(malignant))

Total examples:  10909
Number of benign examples 5480
Number of malignant examples 5429


In [5]:
def load_and_resize_images(img_list, label, size=(128, 128)):
    img_array = []
    for img in img_list:
        image = cv2.imread(img, cv2.IMREAD_COLOR)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        image = tf.convert_to_tensor(image, dtype=tf.float32) / 255.0

        # Get augmented images based on label
        if label in [0, 1, 2, 3]:
            augmented_images = augment_image(image, times=2)
        elif label in [5, 6, 7]:
            augmented_images = augment_image(image, times=3)
        else:  # label == 4
            augmented_images = [image]

        for aug_img in augmented_images:
            image_resized = tf.image.resize(aug_img, size)
            img_array.append([image_resized, label])

    return img_array

def augment_image(img, times=1):
    """
    Augments the image to return `times` total images (including original).
    """
    augmented_images = [img]  # include the original image

    if times > 1:
        # Create augmentations (rotations + flip)
        rotations = [tf.image.rot90(img, k=i) for i in range(1, 4)]
        flips = [tf.image.flip_left_right(r) for r in rotations]
        all_augments = rotations + flips

        # Select needed number of augmentations
        needed = times - 1
        augmented_images.extend(all_augments[:needed])

    return augmented_images

# Load all image sets
A_imgs  = load_and_resize_images(A, 0)
F_imgs  = load_and_resize_images(F, 1)
PT_imgs = load_and_resize_images(PT, 2)
TA_imgs = load_and_resize_images(TA, 3)
DC_imgs = load_and_resize_images(DC, 4)
LC_imgs = load_and_resize_images(LC, 5)
MC_imgs = load_and_resize_images(MC, 6)
PC_imgs = load_and_resize_images(PC, 7)


I0000 00:00:1745388402.984603      31 gpu_device.cc:2022] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 15513 MB memory:  -> device: 0, name: Tesla P100-PCIE-16GB, pci bus id: 0000:00:04.0, compute capability: 6.0


In [6]:
data_b = A_imgs + F_imgs + PT_imgs + TA_imgs 
data_m = DC_imgs + LC_imgs + MC_imgs + PC_imgs
print("Total number of benign examples:", len(data_b))
print("Total number of malignant examples:", len(data_m))

random.shuffle(data_b)
random.shuffle(data_m)

# Initial split to separate out the test set (20% of the data)
dataTrain_b, dataTest_b = train_test_split(data_b, test_size=0.2, random_state=42)

# Further split the training set into training and validation sets (75% train, 25% val of 80% train-val data)
dataTrain_b, dataVal_b = train_test_split(dataTrain_b, test_size=0.25, random_state=42)

# Initial split to separate out the test set (20% of the data)
dataTrain_m, dataTest_m = train_test_split(data_m, test_size=0.2, random_state=42)

# Further split the training set into training and validation sets (75% train, 25% val of 80% train-val data)
dataTrain_m, dataVal_m = train_test_split(dataTrain_m, test_size=0.25, random_state=42)

dataTrain_b_modified = [(image, 0) for image, label in dataTrain_b]
dataTrain_m_modified = [(image, 1) for image, label in dataTrain_m]

dataTrain = dataTrain_b_modified + dataTrain_m_modified
random.shuffle(dataTrain)

dataVal_b_modified = [(image, 0) for image, label in dataVal_b]
dataVal_m_modified = [(image, 1) for image, label in dataVal_m]

dataVal = dataVal_b_modified + dataVal_m_modified
random.shuffle(dataVal)

dataTest_b_modified = [(image, 0) for image, label in dataTest_b]
dataTest_m_modified = [(image, 1) for image, label in dataTest_m]

dataTest = dataTest_b_modified + dataTest_m_modified

print("Total number of examples:", len(dataTrain) + len(dataVal) + len(dataTest))

Total number of benign examples: 10960
Total number of malignant examples: 9385
Total number of examples: 20345


In [7]:
def create_tf_dataset(img_list, batch_size=64):
    images, labels = zip(*img_list)
    images = tf.stack(images)  # More consistent than np.array when using tensors
    labels = tf.convert_to_tensor(labels)

    # One-hot encode labels (binary classification)
    labels = tf.keras.utils.to_categorical(labels, num_classes=2)

    dataset = tf.data.Dataset.from_tensor_slices((images, labels))
    dataset = dataset.shuffle(buffer_size=1000).batch(batch_size).prefetch(tf.data.AUTOTUNE)

    return dataset

# Create datasets
train_dataset = create_tf_dataset(dataTrain)
val_dataset = create_tf_dataset(dataVal)
test_dataset = create_tf_dataset(dataTest)


In [8]:
import tensorflow as tf
from tensorflow.keras.utils import plot_model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Flatten, Dropout, Lambda, BatchNormalization, UnitNormalization
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau

In [9]:
# Base Model
base_model = tf.keras.applications.DenseNet121(
    weights='imagenet',
    include_top=False,
    input_shape=(128, 128, 3)
)
# base_model.trainable = False
# Extract intermediate layers
layer_names = ['conv3_block12_concat', 'conv4_block24_concat', 'conv5_block16_concat']
intermediate_outputs = [base_model.get_layer(name).output for name in layer_names]

# Create a new model that outputs intermediate layers
intermediate_model = Model(inputs=base_model.input, outputs=intermediate_outputs)

# Define the branches for each intermediate output
branch_outputs = []
for output in intermediate_outputs:
    x = GlobalAveragePooling2D()(output)
    x = UnitNormalization(axis=-1)(x)
    x = Dense(64, activation='relu', kernel_regularizer=tf.keras.regularizers.l2(0.001))(x)
    x = BatchNormalization()(x)
    x = Dense(32, activation='relu', kernel_regularizer=tf.keras.regularizers.l2(0.001))(x)
    x = BatchNormalization()(x)
    branch_outputs.append(x)
# Concatenate the branch outputs for score-level fusion
fusion = tf.keras.layers.Concatenate()(branch_outputs)

# Final dense layers
x = Dense(16, activation='relu', kernel_regularizer=tf.keras.regularizers.l2(0.001))(fusion)
x = BatchNormalization()(x)
x = Dropout(0.45)(x)
final_output = Dense(2, activation='softmax')(x)

# Combine everything into a model
binary_model = Model(inputs=base_model.input, outputs=final_output)

optimizer = Adam(learning_rate=0.0002)

binary_model.compile(optimizer=optimizer,
                     loss='categorical_crossentropy',
                     metrics=['accuracy'])

# Callbacks for early stopping and learning rate scheduling
early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=3, min_lr=1e-6)

29084464/29084464 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


In [10]:
import time

start_time = time.time()

history = binary_model.fit(train_dataset,
                           validation_data=val_dataset,
                           epochs=60,
                           callbacks=[early_stopping, reduce_lr])

end_time = time.time()

total_time = end_time - start_time
print(f"Total time taken to train the model: {total_time:.2f} seconds")

Epoch 1/60


I0000 00:00:1745388776.331113      92 service.cc:148] XLA service 0x798948002050 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1745388776.332008      92 service.cc:156]   StreamExecutor device (0): Tesla P100-PCIE-16GB, Compute Capability 6.0
I0000 00:00:1745388785.537044      92 cuda_dnn.cc:529] Loaded cuDNN version 90300
I0000 00:00:1745388857.634088      92 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


191/191 ━━━━━━━━━━━━━━━━━━━━ 304s 669ms/step - accuracy: 0.8263 - loss: 0.8938 - val_accuracy: 0.4613 - val_loss: 1.3058 - learning_rate: 2.0000e-04
Epoch 2/60
191/191 ━━━━━━━━━━━━━━━━━━━━ 25s 128ms/step - accuracy: 0.9483 - loss: 0.5924 - val_accuracy: 0.9098 - val_loss: 0.7246 - learning_rate: 2.0000e-04
Epoch 3/60
191/191 ━━━━━━━━━━━━━━━━━━━━ 24s 128ms/step - accuracy: 0.9667 - loss: 0.5019 - val_accuracy: 0.9639 - val_loss: 0.4979 - learning_rate: 2.0000e-04
Epoch 4/60
191/191 ━━━━━━━━━━━━━━━━━━━━ 25s 128ms/step - accuracy: 0.9736 - loss: 0.4470 - val_accuracy: 0.9558 - val_loss: 0.4693 - learning_rate: 2.0000e-04
Epoch 5/60
191/191 ━━━━━━━━━━━━━━━━━━━━ 24s 127ms/step - accuracy: 0.9825 - loss: 0.3884 - val_accuracy: 0.7793 - val_loss: 1.1933 - learning_rate: 2.0000e-04
Epoch 6/60
191/191 ━━━━━━━━━━━━━━━━━━━━ 25s 129ms/step - accuracy: 0.9808 - loss: 0.3579 - val_accuracy: 0.9629 - val_loss: 0.3981 - learning_rate: 2.0000e-04
Epoch 7/60
191/191 ━━━━━━━━━━━━━━━━━━━━ 24s 128ms/step -

In [11]:
test_loss, test_acc = binary_model.evaluate(test_dataset)
print(f"Test accuracy (binary classification): {test_acc}")

64/64 ━━━━━━━━━━━━━━━━━━━━ 3s 34ms/step - accuracy: 0.9839 - loss: 0.1038
Test accuracy (binary classification): 0.9869747161865234


In [12]:
binary_model.save('binarybreast_model.h5')

**BENIGN MODEL**

In [13]:
dataTrain_m = [(image, label-4) for image, label in dataTrain_m]
dataVal_m = [(image, label-4) for image, label in dataVal_m]
dataTest_m = [(image, label-4) for image, label in dataTest_m]

In [14]:
def create_tf_dataset(img_list, batch_size=64):
    images, labels = zip(*img_list)
    images = np.array(images)  # Convert images to numpy array
    labels = np.array(labels)
    
    # One-hot encode labels
    labels = tf.keras.utils.to_categorical(labels, num_classes=4)
    
    dataset = tf.data.Dataset.from_tensor_slices((images, labels))
    dataset = dataset.shuffle(buffer_size=1000).batch(batch_size).prefetch(tf.data.AUTOTUNE)
    
    return dataset

dataTrain_b = create_tf_dataset(dataTrain_b)
dataVal_b = create_tf_dataset(dataVal_b)
test_dataset_b = create_tf_dataset(dataTest_b)
dataTrain_m = create_tf_dataset(dataTrain_m)
dataVal_m = create_tf_dataset(dataVal_m)
test_dataset_m = create_tf_dataset(dataTest_m)

In [15]:
'''import tensorflow as tf
from tensorflow.keras import Model
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense, BatchNormalization, Dropout, Concatenate, 
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau'''

# Base model with DenseNet121
base_model_b = tf.keras.applications.DenseNet121(
    weights='imagenet',
    include_top=False,
    input_shape=(128, 128, 3)
)

# Extract intermediate layers
layer_names = ['conv3_block12_concat', 'conv4_block24_concat', 'conv5_block16_concat']
intermediate_outputs = [base_model_b.get_layer(name).output for name in layer_names]

# Create intermediate model
intermediate_model = Model(inputs=base_model_b.input, outputs=intermediate_outputs)

# Define the branches for each intermediate output
branch_outputs = []
for output in intermediate_outputs:
    x = GlobalAveragePooling2D()(output)
    x = UnitNormalization(axis=-1)(x)  # Replaces Lambda with UnitNormalization
                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                    x = Dense(64, activation='relu', kernel_regularizer=tf.keras.regularizers.l2(0.001))(x)
    x = BatchNormalization()(x)
    x = Dense(32, activation='relu', kernel_regularizer=tf.keras.regularizers.l2(0.001))(x)
    x = BatchNormalization()(x)
    branch_outputs.append(x)

# Concatenate the branch outputs for score-level fusion
fusion = tf.keras.layers.Concatenate()(branch_outputs)

# Final dense layers
x = Dense(16, activation='relu', kernel_regularizer=tf.keras.regularizers.l2(0.001))(fusion)
x = BatchNormalization()(x)
x = Dropout(0.45)(x)
final_output = Dense(4, activation='softmax')(x)

# Combine everything into a model
benign_model = Model(inputs=base_model_b.input, outputs=final_output)

# Compile the model
optimizer = Adam(learning_rate=0.0001)

benign_model.compile(optimizer=optimizer,
                     loss='categorical_crossentropy',
                     metrics=['accuracy'])

# Callbacks for early stopping and learning rate scheduling
early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=3, min_lr=1e-6)

# Model summary
#benign_model.summary()


In [16]:
import time

start_time = time.time()

history_b = benign_model.fit(dataTrain_b,
                    validation_data=dataVal_b,
                    epochs=120,callbacks=[early_stopping,reduce_lr])

end_time = time.time()

total_time = end_time - start_time
print(f"Total time taken to train the model: {total_time:.2f} seconds")

Epoch 1/120
103/103 ━━━━━━━━━━━━━━━━━━━━ 284s 1s/step - accuracy: 0.4849 - loss: 1.8703 - val_accuracy: 0.3814 - val_loss: 1.8566 - learning_rate: 1.0000e-04
Epoch 2/120
103/103 ━━━━━━━━━━━━━━━━━━━━ 13s 130ms/step - accuracy: 0.7950 - loss: 1.0520 - val_accuracy: 0.5082 - val_loss: 1.7576 - learning_rate: 1.0000e-04
Epoch 3/120
103/103 ━━━━━━━━━━━━━━━━━━━━ 13s 131ms/step - accuracy: 0.8709 - loss: 0.8932 - val_accuracy: 0.6930 - val_loss: 1.5225 - learning_rate: 1.0000e-04
Epoch 4/120
103/103 ━━━━━━━━━━━━━━━━━━━━ 13s 131ms/step - accuracy: 0.9152 - loss: 0.7867 - val_accuracy: 0.8577 - val_loss: 1.1983 - learning_rate: 1.0000e-04
Epoch 5/120
103/103 ━━━━━━━━━━━━━━━━━━━━ 13s 130ms/step - accuracy: 0.9331 - loss: 0.7310 - val_accuracy: 0.8805 - val_loss: 0.9660 - learning_rate: 1.0000e-04
Epoch 6/120
103/103 ━━━━━━━━━━━━━━━━━━━━ 13s 131ms/step - accuracy: 0.9535 - loss: 0.6710 - val_accuracy: 0.9348 - val_loss: 0.7167 - learning_rate: 1.0000e-04
Epoch 7/120
103/103 ━━━━━━━━━━━━━━━━━━━━ 1

In [17]:
test_loss, test_acc = benign_model.evaluate(test_dataset_b)
print(f"Test accuracy -multi class for benign-: {test_acc:.4f}")

35/35 ━━━━━━━━━━━━━━━━━━━━ 2s 36ms/step - accuracy: 0.9703 - loss: 0.3025
Test accuracy -multi class for benign-: 0.9735


In [18]:
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score

In [19]:
y_true_b = []
y_pred_b = []

for images, labels in test_dataset_b:  
    # Predict the classes for each image
    predictions_b = benign_model.predict(images, verbose=0)
    predicted_classes_b = np.argmax(predictions_b, axis=1) 
    true_classes_b = np.argmax(labels, axis=1) 

    y_true_b.extend(true_classes_b)
    y_pred_b.extend(predicted_classes_b)

y_true_b = np.array(y_true_b)
y_pred_b = np.array(y_pred_b)

precision = precision_score(y_true_b, y_pred_b, average='macro')
recall = recall_score(y_true_b, y_pred_b, average='macro')
f1 = f1_score(y_true_b, y_pred_b, average='macro')

print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")
print(f"F1 Score: {f1:.4f}")

Precision: 0.9757
Recall: 0.9728
F1 Score: 0.9741


In [20]:
binary_model.save('breast_BENIGN_model.h5')

**MALIGNANT**

In [21]:
'''import tensorflow as tf
from tensorflow.keras import Model
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense, BatchNormalization, Dropout, Concatenate, 
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau'''

# Base model with DenseNet121
base_model_m = tf.keras.applications.DenseNet121(
    weights='imagenet',
    include_top=False,
    input_shape=(128, 128, 3)
)

# Extract intermediate layers
layer_names = ['conv3_block12_concat', 'conv4_block24_concat', 'conv5_block16_concat']
intermediate_outputs = [base_model_b.get_layer(name).output for name in layer_names]

# Create intermediate model
intermediate_model = Model(inputs=base_model_b.input, outputs=intermediate_outputs)

# Define the branches for each intermediate output
branch_outputs = []
for output in intermediate_outputs:
    x = GlobalAveragePooling2D()(output)
    x = UnitNormalization(axis=-1)(x)  # Replaces Lambda with UnitNormalization
    x = Dense(64, activation='relu', kernel_regularizer=tf.keras.regularizers.l2(0.001))(x)
    x = BatchNormalization()(x)
    x = Dense(32, activation='relu', kernel_regularizer=tf.keras.regularizers.l2(0.001))(x)
    x = BatchNormalization()(x)
    branch_outputs.append(x)

# Concatenate the branch outputs for score-level fusion
fusion = tf.keras.layers.Concatenate()(branch_outputs)

# Final dense layers
x = Dense(16, activation='relu', kernel_regularizer=tf.keras.regularizers.l2(0.001))(fusion)
x = BatchNormalization()(x)
x = Dropout(0.45)(x)
final_output = Dense(4, activation='softmax')(x)

# Combine everything into a model
malignant_model = Model(inputs=base_model_b.input, outputs=final_output)

# Compile the model
optimizer = Adam(learning_rate=0.0001)

malignant_model.compile(optimizer=optimizer,
                     loss='categorical_crossentropy',
                     metrics=['accuracy'])

# Callbacks for early stopping and learning rate scheduling
early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=3, min_lr=1e-6)

# Model summary
#malignant_model.summary()


In [22]:
import time

start_time = time.time()

history_m = malignant_model.fit(dataTrain_m,
                                validation_data=dataVal_m,
                                epochs=50)

end_time = time.time()

total_time = end_time - start_time
print(f"Total time taken to train the model: {total_time:.2f} seconds")

Epoch 1/50
88/88 ━━━━━━━━━━━━━━━━━━━━ 280s 1s/step - accuracy: 0.4478 - loss: 1.9730 - val_accuracy: 0.3671 - val_loss: 1.8333
Epoch 2/50
88/88 ━━━━━━━━━━━━━━━━━━━━ 11s 129ms/step - accuracy: 0.7041 - loss: 1.2484 - val_accuracy: 0.4465 - val_loss: 1.7518
Epoch 3/50
88/88 ━━━━━━━━━━━━━━━━━━━━ 11s 128ms/step - accuracy: 0.7923 - loss: 1.0283 - val_accuracy: 0.6574 - val_loss: 1.6169
Epoch 4/50
88/88 ━━━━━━━━━━━━━━━━━━━━ 11s 128ms/step - accuracy: 0.8498 - loss: 0.9170 - val_accuracy: 0.8247 - val_loss: 1.4033
Epoch 5/50
88/88 ━━━━━━━━━━━━━━━━━━━━ 11s 128ms/step - accuracy: 0.8940 - loss: 0.8292 - val_accuracy: 0.8466 - val_loss: 1.1537
Epoch 6/50
88/88 ━━━━━━━━━━━━━━━━━━━━ 11s 127ms/step - accuracy: 0.9216 - loss: 0.7569 - val_accuracy: 0.8807 - val_loss: 0.9654
Epoch 7/50
88/88 ━━━━━━━━━━━━━━━━━━━━ 11s 129ms/step - accuracy: 0.9296 - loss: 0.7267 - val_accuracy: 0.9105 - val_loss: 0.7809
Epoch 8/50
88/88 ━━━━━━━━━━━━━━━━━━━━ 11s 128ms/step - accuracy: 0.9514 - loss: 0.6801 - val_accura

In [23]:
test_loss, test_acc = malignant_model.evaluate(test_dataset_m)
print(f"Test accuracy -multi class for malignant-: {test_acc:.4f}")

30/30 ━━━━━━━━━━━━━━━━━━━━ 1s 34ms/step - accuracy: 0.8449 - loss: 0.8514
Test accuracy -multi class for malignant-: 0.8519


In [24]:
malignant_model.save('breast_malignant_model.h5')